# Extract CERFA variables from the tax simulator

In [31]:
from bs4 import BeautifulSoup
import requests
import re
from collections import defaultdict
from tqdm.notebook import tqdm
import json

from deuxpots import CERFA_VARIABLES_PATH

In [32]:
SIMULATOR_HTML_URL = 'https://simulateur-ir-ifi.impots.gouv.fr/calcul_impot/2023/complet/index.htm'

## Get simulator HTML page

In [33]:
response = requests.get(SIMULATOR_HTML_URL)
soup = BeautifulSoup(response.text)

In [34]:
REGEX_WHITESPACES = re.compile(r'\s+')
def clean_text(text):
    return REGEX_WHITESPACES.sub(' ', text).strip().strip('-:– ')

## Extract boxes

Iterate over `input` fields and look at their `aria-labelledby` property to get their description.

In [35]:
BOXCODE_REGEX = re.compile(r'^[0-9A-Z]{2,3}$')    
USE_TITLE_FOR = {"5VA", "7JP"}

input_fields = soup.find_all('input', type=['text', 'checkbox', 'radio'])

boxcodes_missing_chunks = set()

boxes = []
for field in tqdm(input_fields):
    # 0. Preprocessing
    labelled_by = field.get('aria-labelledby')
    if not labelled_by:
        continue
    ref_ids = labelled_by.split()
    if not ref_ids[0] == 'case':
        continue

    # 1. Extract box code
    field_title = clean_text(field['title'])
    if field_title in USE_TITLE_FOR:
        boxcode = field_title
    else:
        ref_ids.pop(0)
        ref_id_box = ref_ids.pop(0)
        boxcode_tag = field.find_previous(id=ref_id_box)
        if not boxcode_tag:
            # For checkboxes, the label is after the box (on the right hand side)
            boxcode_tag = field.find_next(id=ref_id_box) 
        if boxcode_tag:
            boxcode = clean_text(boxcode_tag.text)
        else:
            boxcode = field_title
    if not boxcode:
        print(f"Not found: {ref_id_box}\t", labelled_by)
        continue
    if not BOXCODE_REGEX.match(boxcode):
        print("Weird boxcode: ", boxcode, ref_ids)
    
    # 2. Extract description chunks
    ref_ids.remove('pause')
    section_tag = field.find_previous(lambda tag: tag.get('class') == ['bgCommonDarkPart'] and tag.find('strong'))
    section_text = clean_text(section_tag.text)
    description_chunks = []
    for i, ref_id in enumerate(ref_ids):
        ref_tag = field.find_previous(id=ref_id)
        # When the tag if found after the box, it is actually a mistake in the simulator.
        # (i.e. the tag should not be there). So it is best to skip it.
        if not ref_tag:
            boxcodes_missing_chunks.add(boxcode)
            continue
        ref_text = clean_text(ref_tag.text)
        # Prepend the big section title if not already referenced in "aria-labelledby"
        if i == 0 and section_text != ref_text:
            description_chunks.append(section_text)
        description_chunks.append(ref_text)
    boxes.append({
        'code': boxcode,
        'description_chunks': description_chunks,
        'type': 'int' if field['type'] == 'text' else 'bool'
    })

  0%|          | 0/1438 [00:00<?, ?it/s]

In [36]:
boxes_by_code = {box['code']: box for box in boxes}

## Check duplicates

In [37]:
boxes_counts = defaultdict(int)
for box in boxes:
    boxes_counts[box['code']] += 1

for box in boxes:
    if boxes_counts[box['code']] > 1:
        print(box['code'], box['description'], box['attribution'])

## Add attributions (from chunks)

In [38]:
ATTRIBUTION_FIELDS = {'Déclarant 1', 'Déclarant 2', 'Personne à charge', '1ère personne à charge', '2ème personne à charge'}
for box in boxes:
    chunks = box['description_chunks']
    if chunks and chunks[-1] in ATTRIBUTION_FIELDS:
        box['attribution'] = chunks.pop()
    else:
        box['attribution'] = None

## Manually override some mislabelled fields

In [39]:
MANUAL_FIELDS_OVERRIDE = {
    '1AV': {'attribution': 'Déclarant 1'},
    '1BV': {'attribution': 'Déclarant 2'},
    '1CH': {'attribution': '1ère personne à charge'},
    '1DH': {'attribution': '2ème personne à charge'},
    '5HH': {'description_chunks': boxes_by_code['5IH']['description_chunks']},
    '5VJ': {'description_chunks': boxes_by_code['5UJ']['description_chunks']},
    '1BH': {'attribution': 'Déclarant 2'},
    '8HV': {'attribution': '2ème personne à charge'},
    '8HW': {'attribution': 'Déclarant 1'},
    '8HX': {'attribution': 'Déclarant 1'},
    '8HY': {'attribution': 'Déclarant 1'},
    '8HZ': {'attribution': 'Déclarant 1'},
    '8SI': {'attribution': 'Déclarant 2'},
    '5UC': {'attribution': 'Déclarant 2'},
    '5VC': {'attribution': 'Personne à charge'},
    '8HV': {'attribution': 'Déclarant 1'},

    '5TA': {'description_chunks': boxes_by_code['5TA']['description_chunks'][0:-1] + ['Ventes de marchandises et assimilées'] + boxes_by_code['5TA']['description_chunks'][-1:]},
    '5UA': {'description_chunks': boxes_by_code['5TA']['description_chunks'][0:-1] + ['Ventes de marchandises et assimilées'] + boxes_by_code['5TA']['description_chunks'][-1:]},
    '5VA': {'description_chunks': boxes_by_code['5TA']['description_chunks'][0:-1] + ['Ventes de marchandises et assimilées'] + boxes_by_code['5TA']['description_chunks'][-1:]},
    
    '5TJ': {'description_chunks': boxes_by_code['5TJ']['description_chunks'][0:-1] + ['Ventes de marchandises et assimilées'] + boxes_by_code['5TJ']['description_chunks'][-1:]},
    '5UJ': {'description_chunks': boxes_by_code['5TJ']['description_chunks'][0:-1] + ['Ventes de marchandises et assimilées'] + boxes_by_code['5TJ']['description_chunks'][-1:]},
    '5VJ': {'description_chunks': boxes_by_code['5TJ']['description_chunks'][0:-1] + ['Ventes de marchandises et assimilées'] + boxes_by_code['5TJ']['description_chunks'][-1:]},
    
    '5TB': {'description_chunks': boxes_by_code['5TB']['description_chunks'][0:-1] + ['Prestations de services et locations meublées'] + boxes_by_code['5TB']['description_chunks'][-1:]},
    '5UB': {'description_chunks': boxes_by_code['5TB']['description_chunks'][0:-1] + ['Prestations de services et locations meublées'] + boxes_by_code['5TB']['description_chunks'][-1:]},
    '5VB': {'description_chunks': boxes_by_code['5TB']['description_chunks'][0:-1] + ['Prestations de services et locations meublées'] + boxes_by_code['5TB']['description_chunks'][-1:]},
    
    '5TK': {'description_chunks': boxes_by_code['5TK']['description_chunks'][0:-1] + ['Prestations de services et locations meublées'] + boxes_by_code['5TK']['description_chunks'][-1:]},
    '5UK': {'description_chunks': boxes_by_code['5TK']['description_chunks'][0:-1] + ['Prestations de services et locations meublées'] + boxes_by_code['5TK']['description_chunks'][-1:]},
    '5VK': {'description_chunks': boxes_by_code['5TK']['description_chunks'][0:-1] + ['Prestations de services et locations meublées'] + boxes_by_code['5TK']['description_chunks'][-1:]},

    '5AF': {'description_chunks': boxes_by_code['5AD']['description_chunks'][0:] + boxes_by_code['5AF']['description_chunks']},
    '5AI': {'description_chunks': boxes_by_code['5AD']['description_chunks'][0:] + boxes_by_code['5AF']['description_chunks']},
    '5AH': {'description_chunks': boxes_by_code['5AD']['description_chunks'][0:] + boxes_by_code['5AF']['description_chunks']},

    '6QS': {'description_chunks': ["Autres cotisations déduites des BIC, BNC, BA, rémun. art 62 CGI ou salaires"] + boxes_by_code['6QS']['description_chunks']},
    '6QT': {'description_chunks': ["Autres cotisations déduites des BIC, BNC, BA, rémun. art 62 CGI ou salaires"] + boxes_by_code['6QS']['description_chunks']},
    '6QU': {'description_chunks': ["Autres cotisations déduites des BIC, BNC, BA, rémun. art 62 CGI ou salaires"] + boxes_by_code['6QS']['description_chunks']},
    
    '7WC': {'description_chunks': [
        "PROROGATION EN 2022 DE L'ENGAGEMENT DE LOCATION DANS LE SECTEUR INTERMÉDIAIRE",
 	    "Investissements en métropole et dans les DOM-COM",
        "Investissements achevés en 2013",
        "Investissements réalisés en 2013",
        "Investissement réalisé du 1.1 au 31.3.2013 avec contrat de réservation enregistré au plus tard le 31/12/2012",
    ]},

    '7BK': {'description_chunks': boxes_by_code['7BK']['description_chunks'] + ["15%"]},
    '7BL': {'description_chunks': boxes_by_code['7BL']['description_chunks'] + ["20%"]},
    '7BM': {'description_chunks': boxes_by_code['7BM']['description_chunks'] + ["35%"]},
    '7BN': {'description_chunks': boxes_by_code['7BN']['description_chunks'] + ["40%"]},
    '7BO': {'description_chunks': boxes_by_code['7BO']['description_chunks'] + ["65%"]},
    
    '7ZQ': {'description_chunks': boxes_by_code['7ZQ']['description_chunks'] + ["Dans l'habitation principale", "1er système"]},
    '7ZR': {'description_chunks': boxes_by_code['7ZR']['description_chunks'] + ["Dans l'habitation principale", "2eme système"]},
    '7ZS': {'description_chunks': boxes_by_code['7ZS']['description_chunks'] + ["Dans l'habitation secondaire", "1er système"]},
    '7ZT': {'description_chunks': boxes_by_code['7ZT']['description_chunks'] + ["Dans l'habitation secondaire", "2eme système"]},

    '7TK': {'description_chunks': boxes_by_code['7TK']['description_chunks'][0:2] + boxes_by_code['7TJ']['description_chunks'][-2:-1] + boxes_by_code['7TK']['description_chunks'][2:]},
    '7TO': {'description_chunks': boxes_by_code['7TO']['description_chunks'][0:2] + boxes_by_code['7TM']['description_chunks'][-2:-1] + boxes_by_code['7TO']['description_chunks'][2:]},
    '7TQ': {'description_chunks': boxes_by_code['7TQ']['description_chunks'][0:2] + boxes_by_code['7TP']['description_chunks'][-2:-1] + boxes_by_code['7TQ']['description_chunks'][2:]},
    '7TS': {'description_chunks': boxes_by_code['7TS']['description_chunks'][0:2] + boxes_by_code['7TR']['description_chunks'][-2:-1] + boxes_by_code['7TS']['description_chunks'][2:]},

    '5ZW': {'description_chunks': boxes_by_code['5XZ']['description_chunks']},
}


for boxcode, fields_map in MANUAL_FIELDS_OVERRIDE.items():
    for field_name, field_value in fields_map.items():
        boxes_by_code[boxcode][field_name] = field_value

## Add description field

In [40]:
for box in boxes:
    box['description'] = ' - '.join(box['description_chunks'])

## Visual checks

### Inspect the description for the fields where some labels were not found  

In [41]:
for code in boxcodes_missing_chunks:
    print(code, boxes_by_code[code]['description'])

7IG VOS CHARGES OUVRANT DROIT A RÉDUCTION D'IMPÔT OU A CREDIT D'IMPÔT - Investissements achevés en 2016 - Investissements réalisés en 2012
7KO Investissements réalisés en 2012 - Engagement de réalisation de l’investissement en 2012
HIU AUTRES CHARGES OUVRANT DROIT A RÉDUCTION D'IMPÔT : Investissements outre-mer - RÉDUCTION D’IMPÔT POUR INVESTISSEMENTS OUTRE-MER DANS LE CADRE D’UNE ENTREPRISE - Investissements réalisés en 2022 - Investissements dans votre entreprise
7WY Investissements achevés en 2013 en Polynésie française, Nouvelle Calédonie, dans les îles Wallis et Futuna - Investissements réalisés en 2011 ou réalisés en 2012 avec promesse d’achat en 2011
7CH VOS CHARGES OUVRANT DROIT A RÉDUCTION D'IMPÔT OU A CREDIT D'IMPÔT - Souscription au capital de PME, d’entreprises d’utilité sociale et de sociétés foncières et solidaires - Versements PME et ESUS effectués du 18.03 au 31.12.2022
6QS Autres cotisations déduites des BIC, BNC, BA, rémun. art 62 CGI ou salaires - VOS CHARGES ET IMPU

### Inspect boxes with very short description

In [42]:
for box in boxes:
    if len(box['description_chunks']) <= 1:
        print(box['code'], box['description'])

0XX REVENUS EXCEPTIONNELS OU DIFFERES A IMPOSER SELON LE SYSTEME DU QUOTIENT
7WE Engagement de réalisation de l’investissement en 2011
7YY Investissements réalisés en 2010 avec promesse d'achat en 2009
7YX Investissements réalisés en 2010
7YZ Investissements réalisés en 2009
7KS Investissements réalisés du 1.1.2013 au 31.3.2013 avec engagement de réalisation en 2012
7BF Engagement de réalisation de l’investissement en 2011


## Pregroup boxes by description

In [43]:
boxes_pregrouped = defaultdict(list)
for box in boxes:
    boxes_pregrouped[box['description']].append(box)

## Group boxes into variables
Boxes are grouped together if they are attributed and they have the same description.

In [44]:
variables = []
for description, box_group in boxes_pregrouped.items():
    # Check attributions
    is_decl1 = [box for box in box_group if box['attribution'] == "Déclarant 1"]
    is_decl2 = [box for box in box_group if box['attribution'] == "Déclarant 2"]
    is_attributed = [box for box in box_group if box['attribution']]
    types = {box['type'] for box in box_group}
    if (len(is_decl1) > 1
        or len(is_decl2) > 1
        or len(is_attributed) / len(box_group) not in (0., 1.)
        or is_attributed and box_group[0]['attribution'] != "Déclarant 1"
        or is_attributed and box_group[1]['attribution'] != "Déclarant 2"
        or len(types) != 1):
        print(description)
        for box in box_group:
            print(box['code'], box['attribution'])
        print('\n')

    # Group attributed boxes into variables
    var_type = types.pop()
    if is_attributed:
       variables.append({
            'boxes': [box['code'] for box in box_group],
            'description': description,
            'type': var_type,
        })
    else:
        for box in box_group:
            variables.append({
                'boxes': [box['code']],
                'description': description,
                'type': var_type,
            })

## Manually add some boxes

### Family boxes

In [45]:
variables += [
    # The order is important here: the first one is the box for the first partner (will be always used for single simulation)
    {'boxes': ['0AP', '0AF'], 'type': 'bool', 'description': "Titulaire d'une pension pour une invalidité d'au moins 40 % ou d'une carte d'invalidité d'au moins 80%"},
    {'boxes': ['0AW', '0AS'], 'type': 'bool', 'description': "Vous êtes âgé de plus de 74 ans, vous êtes titulaire de la carte du combattant ou d'une pension militaire d'invalidité ou de victime de guerre"},
    {'boxes': ['0CF',], 'type': 'float', 'description': "Nombre d'enfants à charge non mariés, qui ne sont pas en résidence alternée, de moins de 18 ans au 1er janvier de l'année de perception des revenus, ou nés durant la même année ou handicapés quel que soit leur âge"},
    {'boxes': ['0CG',], 'type': 'float', 'description': "Nombre d'enfants qui ne sont pas en résidence alternée à charge titulaires de la carte d'invalidité."},
    {'boxes': ['0CH',], 'type': 'float', 'description': "Nombre d'enfants à charge en résidence alternée, non mariés de moins de 18 ans au 1er janvier de l'année de perception des revenus, ou nés durant la même année ou handicapés quel que soit leur âge"},
    {'boxes': ['0CI',], 'type': 'float', 'description': "Nombre d'enfants à charge en résidence alternée titulaires de la carte d'invalidité"},
    {'boxes': ['0CR',], 'type': 'float', 'description': "Nombre de titulaires (autres que les enfants) de la carte invalidité d'au moins 80 %"},
    {'boxes': ['0DJ',], 'type': 'float', 'description': "Nombre d'enfants majeurs célibataires sans enfant"},
    {'boxes': ['0DN',], 'type': 'float', 'description': "Nombre d'enfants mariés/pacsés et d'enfants non mariés chargés de famille"},
]

### Household status boxes

In [46]:
variables += [
    {'boxes': ['0AM'], 'type': 'bool', 'description': "Marié·e"},
    {'boxes': ['0AO'], 'type': 'bool', 'description': "Pacsé·e"},
    {'boxes': ['0AD'], 'type': 'bool', 'description': "Divorcé·e/séparé·e"},
    {'boxes': ['0AC'], 'type': 'bool', 'description': "Célibataire"},
    {'boxes': ['0AV'], 'type': 'bool', 'description': "Veuf·ve"},
]

## Prettify description

In [47]:
WORD_REPLACEMENTS = [
    ('dun', "d'un"),
    ("lemploi", "l'emploi"),
    ('marié', 'marié·e'),
    ('mariés', 'marié·e·s'),
    ('âgé', 'âgé·e'),
    ('âgés', 'âgé·e·s'),
    ('pacsé', 'pacsé·e'),
    ('pacsés', 'pacsé·e·s'),
    ('né', 'né·e'),
    ('nés', 'né·e·s'),
    ('majeurs', 'majeur·e·s'),
    ('handicapé', 'handicapé·e'),
    ('handicapés', 'handicapé·e·s'),
    ('salariés', 'salarié·e·s'),
    ('pensionnés', 'pensionné·e·s'),
    ('un salarié', 'un·e salarié·e'),
    ('dun salarié', 'dun·e salarié·e'),
    ('inventeurs', 'inventeur·euse·s'),
    ('auteurs', 'auteur·ice·s'),
    ('demandeur', 'demandeur·euse'),
    ('inscrit', 'inscrit·e'),    
    ('OGAou', 'OGA ou'),    
]
for var in variables:
    if not var['description']:
        continue
    original_description = var['description']
    for lookup, repl in WORD_REPLACEMENTS:
        var['description'] = re.sub(r"\b%s\b" % lookup, repl, var['description'])
        var['description'] = re.sub(r"\b%s\b" % lookup.capitalize(), repl.capitalize(), var['description'])
    if False and var['description'] != original_description:
        print(original_description)
        print(var['description'], '\n')
    if not var['description'].endswith('.'):
        var['description'] = var['description'] + '.'

## Output variables

In [48]:
with open(CERFA_VARIABLES_PATH, 'w+') as f:
    json.dump(variables, f, indent=2)